In [29]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from tensorflow.python.ops.numpy_ops import np_config
housing = fetch_california_housing()

In [87]:
class MLPLinearRegressor(object):
    def __init__(self, lr=0.001, n_layers=3, n_neurons=3):
        self.graph = tf.Graph()
        self.lr = 0.001
        self.n_layers = n_layers
        self.n_neurons = n_neurons
        self.initialized = False

    @tf.function
    def loss(self):
        return 1/2 * tf.reduce_sum( tf.square((tf.reshape(self.predicted, [-1, 1]) - self.y)))

    def initialize(self, X, y):
          n_samples, n_features = X.shape
          n_output = y.shape[0]
          normal_initiliazer = tf.random_normal_initializer(seed=0, mean=0.0, stddev=1.0)
          XN = tf.keras.utils.normalize(X)
          self.X = tf.constant(XN, name="X", dtype=tf.double)
          self.y = tf.constant(np.array(y).reshape(-1, 1), name="y", dtype=tf.double)

          self.f_weights = tf.Variable(normal_initiliazer([self.n_neurons, n_samples, n_features], dtype=tf.double), name="f_weights", dtype=tf.double, trainable=True )
          self.h_weights = tf.Variable(normal_initiliazer([self.n_layers, self.n_neurons, n_samples, n_output], dtype=tf.double), name="h_weights", dtype=tf.double, trainable=True )
          self.bias = tf.Variable([0.01 for l in range(self.n_layers)], name="bias", dtype=tf.double, trainable=True )
          
          self.activation = tf.TensorArray(tf.double, size=0, dynamic_size=True, clear_after_read=False, tensor_array_name="Activation_Layers")
          self.activation.write(0, [tf.math.sigmoid(tf.matmul(X, tf.transpose(self.f_weights[n])) + self.bias[0]) for n in range(self.n_neurons)]).mark_used()
          for l in range(1,self.n_layers-1):
              self.activation.write(l, [tf.math.sigmoid(tf.matmul(self.activation.read(l-1)[n], self.h_weights[l][n]) + self.bias[l]) for n in range(self.n_neurons)])
          self.activation.write(self.n_layers-1, [tf.math.sigmoid(tf.matmul(self.activation.read(self.n_layers-2)[n], self.h_weights[self.n_layers-1][n]) + self.bias[self.n_layers-1]) for n in range(self.n_neurons)])

          self.predicted = tf.Variable(tf.reduce_mean(tf.math.sigmoid(tf.tensordot(tf.reduce_mean(tf.reduce_mean(self.h_weights[self.n_layers-1], axis=0), axis=0), tf.reduce_mean(tf.reduce_mean(self.activation.read(self.n_layers-1), axis=0), axis=1), axes=0) + tf.reshape(self.bias[self.n_layers-1], [-1, 1])), axis=0), trainable=True, name="Predicted")
          #self.loss = lambda : tf.losses.mean_squared_error(self.predicted, y)
          self.learning_rate = tf.keras.optimizers.schedules.ExponentialDecay(self.lr, 5, 0.85, name="Learning_Rate")
          self.optimizer = tf.keras.optimizers.Adam(self.learning_rate)

          self.vars = [self.f_weights, self.h_weights, self.bias, self.predicted]
          self.initialized = True

    def train(self, y, epochs=75):
        if self.initialized:
          for e in range(epochs):
            with tf.GradientTape() as tape:
                loss = self.loss()
            
            grads = tape.gradient(loss, self.vars)
            self.optimizer.apply_gradients(zip(grads, self.vars))
        self.corrects = tf.equal(self.predicted, y)
        self.accuracy = tf.reduce_mean(tf.cast(self.corrects, tf.float32))

            
X_train, X_test, y_train, y_test = train_test_split(housing.data, housing.target, test_size=0.95)

model = MLPLinearRegressor()
model.initialize(X_train, y_train)
model.train(y_train, epochs=1)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "c:\Users\Utente\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py", line 3373, in run_ast_nodes
    if await self.run_code(code, result, async_=asy):  File "c:\Users\Utente\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)  File "C:\Users\Utente\AppData\Local\Temp\ipykernel_18940\750269629.py", line 54, in <module>
    model.initialize(X_train, y_train)  File "C:\Users\Utente\AppData\Local\Temp\ipykernel_18940\750269629.py", line 28, in initialize
    self.activation.write(l, [tf.math.sigmoid(tf.matmul(self.activation.read(l-1)[n], self.h_weights[l][n]) + self.bias[l]) for n in range(self.

ValueError: No gradients provided for any variable: (['f_weights:0', 'h_weights:0'],). Provided `grads_and_vars` is ((None, <tf.Variable 'f_weights:0' shape=(3, 1032, 8) dtype=float64, numpy=
array([[[ 0.50597862, -0.70111141,  0.45309454, ...,  0.57622276,
         -0.16532128,  0.14021994],
        [-1.33475533, -0.07734598,  1.43766396, ..., -0.0521582 ,
          1.07995859, -0.88625755],
        [-1.02305545,  1.54611159,  1.28781231, ...,  0.14079827,
         -0.33146609,  1.59719274],
        ...,
        [ 0.73296047,  0.0603372 , -2.22900904, ...,  0.34560146,
         -0.20135526,  0.16414816],
        [ 1.53523747, -0.54952954, -1.4167278 , ..., -0.38099626,
          1.34786757, -1.55258931],
        [ 3.05293555,  0.89496054,  1.0255621 , ...,  1.33858007,
          0.10878851,  1.1138124 ]],

       [[ 1.72520435, -0.02254615,  0.4565958 , ...,  0.25351815,
         -1.49348338, -0.02426565],
        [-1.73683138, -3.13296541, -1.0009193 , ..., -0.6623713 ,
         -1.5